In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset

from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import AdamW, get_scheduler

from datasets import load_dataset

from tqdm import tqdm

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(torch.__version__)
print(device)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("distilgpt2")
model = AutoModelForCausalLM.from_pretrained("distilgpt2")

In [ ]:
filename = 'data/clean_29_07_2023.txt'

In [ ]:
dataset = load_dataset('text', data_files={'train': filename})

In [ ]:
def tokenize_and_pad(examples):
    # Tokenize the texts
    output = tokenizer(
        examples["text"], 
        truncation=True, 
        padding='max_length', 
        max_length=512, 
        return_tensors='pt'
    )
    return output

tokenizer.pad_token = tokenizer.eos_token
tokenized_datasets = dataset.map(tokenize_and_pad, batched=True, remove_columns=['text'])

In [ ]:
tokenized_datasets = tokenized_datasets["train"].train_test_split(test_size=0.2)

In [ ]:
def collate_fn(batch):
    input_ids = [item['input_ids'] for item in batch]
    attention_mask = [item['attention_mask'] for item in batch]
    
    return {
        'input_ids': torch.tensor(input_ids),
        'attention_mask': torch.tensor(attention_mask)
    }

In [ ]:
train_dataloader = DataLoader(tokenized_datasets["train"], shuffle=True, batch_size=8, collate_fn=collate_fn)
val_dataloader = DataLoader(tokenized_datasets["test"], shuffle=False, batch_size=8, collate_fn=collate_fn)

In [ ]:
optimizer = AdamW(model.parameters(), lr=5e-5)

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

In [ ]:
def move_batch_to_device(batch, device):
    return {k: v.to(device) for k, v in batch.items()}

In [ ]:
model.to(device)

progress_bar = tqdm(range(num_training_steps))

model.train()

for epoch in range(num_epochs):
    for idx, batch in enumerate(train_dataloader):
        batch = move_batch_to_device(batch, device)
        outputs = model(input_ids=batch['input_ids'], 
                        attention_mask=batch['attention_mask'], 
                        labels=batch['input_ids'])
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    model.eval()

    with torch.no_grad():
        total_loss = 0
        for batch in val_dataloader:
            batch = move_batch_to_device(batch, device)
            outputs = model(**batch)
            total_loss += outputs.loss.item()

        avg_val_loss = total_loss / len(val_dataloader)
        print(f"Validation loss: {avg_val_loss}")

    model.train()

In [ ]:
batch = next(iter(train_dataloader))
print(type(batch['input_ids']))
print(type(batch['attention_mask']))
